# Batch Liver IoU (PNG → NIfTI)
This section iterates over liver segmentation PNGs in `LucaSegmentation/Liver segmentation`, extracts the patient ID and slice index from each filename (`liver seg pat {ID} slice {N}.png`), loads the corresponding `segmentation-{ID}.nii`, and computes IoU for the specified slice. It tests common 2D orientation variants to account for PNG export rotations/flips, then reports per-file results and per-patient mean IoU.

# Interesection over Union

## Liver IoU

In [25]:
# Batch IoU over Liver segmentation PNGs -> NIfTI per patient/slice
import os
import re
import numpy as np
from typing import Dict, Tuple, List

try:
    import nibabel as nib
except ImportError as e:
    raise ImportError("Missing dependency 'nibabel'. Install with: pip install nibabel") from e

try:
    from PIL import Image
except ImportError as e:
    raise ImportError("Missing dependency 'Pillow'. Install with: pip install pillow") from e

# Paths
root = os.getcwd()
seg_dir = os.path.join(root, "data", "Segmentation_Files")
png_dir = os.path.join(root, "LucaSegmentation", "Liver segmentation")
print("Segmentation dir:", seg_dir)
print("PNG dir:", png_dir)

if not os.path.isdir(seg_dir):
    raise FileNotFoundError(f"Segmentation directory not found: {seg_dir}")
if not os.path.isdir(png_dir):
    raise FileNotFoundError(f"PNG directory not found: {png_dir}")

# Helpers
fname_re = re.compile(r"liver\s*seg\s*pat\s*(\d+)\s*slice\s*(\d+)\.png\Z", re.IGNORECASE)

def parse_name(name: str) -> Tuple[int, int]:
    m = fname_re.search(name)
    if not m:
        raise ValueError(f"Filename not in expected format 'liver seg pat <ID> slice <N>.png': {name}")
    return int(m.group(1)), int(m.group(2))

# Binary IoU

def compute_iou(a: np.ndarray, b: np.ndarray) -> float:
    inter = np.sum((a != 0) & (b != 0))
    union = np.sum((a != 0) | (b != 0))
    return float(inter) / float(union) if union > 0 else 0.0

# Orientation variants to try
transforms = {
    "identity": lambda x: x,
    "flipud": np.flipud,
    "fliplr": np.fliplr,
    "transpose": lambda x: x.T,
    "transpose_flipud": lambda x: np.flipud(x.T),
    "transpose_fliplr": lambda x: np.fliplr(x.T),
}

# Cache NIfTI volumes by patient id
vol_cache: Dict[int, np.ndarray] = {}

def load_volume_for_patient(pid: int) -> np.ndarray:
    if pid in vol_cache:
        return vol_cache[pid]
    nii_path = os.path.join(seg_dir, f"segmentation-{pid}.nii")
    if not os.path.exists(nii_path):
        raise FileNotFoundError(f"Missing NIfTI for patient {pid}: {nii_path}")
    img = nib.load(nii_path)
    img = nib.as_closest_canonical(img)
    data = img.get_fdata()
    vol = np.asarray(data, dtype=np.int16)
    if vol.ndim > 3:
        vol = np.squeeze(vol)
    if vol.ndim != 3:
        raise ValueError(f"Unexpected volume shape for patient {pid}: {vol.shape}")
    vol_cache[pid] = vol
    return vol

# Discover PNGs
png_files = [f for f in os.listdir(png_dir) if f.lower().endswith('.png')]
png_files.sort()
print(f"Found {len(png_files)} PNG masks.")

if not png_files:
    print("No PNG files found. Ensure the folder and naming are correct.")

# Iterate and compute IoU per file, aggregate per patient
per_file_results: List[Tuple[int, int, float, str]] = []  # (pid, slice, iou, orientation)
per_patient_scores: Dict[int, List[float]] = {}

for fname in png_files:
    try:
        pid, z = parse_name(fname)
    except ValueError as e:
        print("Skipping:", fname, "->", e)
        continue

    try:
        vol = load_volume_for_patient(pid)
    except Exception as e:
        print(f"Patient {pid}: failed to load volume -> {e}")
        continue

    if z < 0 or z >= vol.shape[2]:
        print(f"Patient {pid} slice {z}: out of bounds for Z={vol.shape[2]}")
        continue

    gt = (vol[:, :, z] >= 1).astype(np.uint8)

    # Load prediction and make binary
    png_path = os.path.join(png_dir, fname)
    with Image.open(png_path) as im:
        pred_arr = np.array(im.convert('L'), dtype=np.uint8)
    pred = (pred_arr >= 128).astype(np.uint8)

    # Resize prediction if needed
    if pred.shape != gt.shape:
        pred = (np.array(Image.fromarray(pred * 255).resize((gt.shape[1], gt.shape[0]), Image.NEAREST)) >= 128).astype(np.uint8)

    # Try orientations and pick best
    ious = {name: compute_iou(gt, fn(pred)) for name, fn in transforms.items()}
    best_name = max(ious, key=ious.get)
    best_iou = ious[best_name]

    per_file_results.append((pid, z, best_iou, best_name))
    per_patient_scores.setdefault(pid, []).append(best_iou)

# Report
print("\nPer-file results (first 20 shown):")
for row in per_file_results[:20]:
    pid, z, iou, orient = row
    print(f"pat {pid:>3} slice {z:>4} -> IoU {iou:.4f} (orient={orient})")

print("\nPer-patient summary:")
for pid in sorted(per_patient_scores):
    vals = per_patient_scores[pid]
    mean_iou = float(np.mean(vals)) if vals else 0.0
    print(f"patient {pid:>3}: {len(vals)} slices, mean IoU {mean_iou:.4f}")


Segmentation dir: /Users/matteogiacomelli/Documents/Fisica_Magistrale/Image ProccesingOriginal/Project LiTS/data/Segmentation_Files
PNG dir: /Users/matteogiacomelli/Documents/Fisica_Magistrale/Image ProccesingOriginal/Project LiTS/LucaSegmentation/Liver segmentation
Found 10 PNG masks.

Per-file results (first 20 shown):
pat   0 slice   62 -> IoU 0.6353 (orient=transpose_flipud)
pat  10 slice  380 -> IoU 0.5774 (orient=transpose_flipud)
pat  12 slice  400 -> IoU 0.8729 (orient=transpose_flipud)
pat  16 slice  410 -> IoU 0.8371 (orient=transpose_flipud)
pat  18 slice  420 -> IoU 0.8877 (orient=transpose_flipud)
pat  24 slice  199 -> IoU 0.8401 (orient=transpose_flipud)
pat  27 slice  495 -> IoU 0.8177 (orient=transpose_flipud)
pat   4 slice  460 -> IoU 0.8129 (orient=transpose_flipud)
pat   6 slice  410 -> IoU 0.6875 (orient=transpose_flipud)
pat   8 slice  476 -> IoU 0.9023 (orient=transpose_flipud)

Per-patient summary:
patient   0: 1 slices, mean IoU 0.6353
patient   4: 1 slices, mea

### Volume Overlapping

In [26]:
print("\nPer-patient Liver VOE summary:")
for pid in sorted(per_patient_scores):
    vals = per_patient_scores[pid]
    mean_iou = float(np.mean(vals)) if vals else 0.0
    print(f"patient {pid:>3}: {len(vals)} slices, mean VOE {1 - mean_iou:.4f}")


Per-patient Liver VOE summary:
patient   0: 1 slices, mean VOE 0.3647
patient   4: 1 slices, mean VOE 0.1871
patient   6: 1 slices, mean VOE 0.3125
patient   8: 1 slices, mean VOE 0.0977
patient  10: 1 slices, mean VOE 0.4226
patient  12: 1 slices, mean VOE 0.1271
patient  16: 1 slices, mean VOE 0.1629
patient  18: 1 slices, mean VOE 0.1123
patient  24: 1 slices, mean VOE 0.1599
patient  27: 1 slices, mean VOE 0.1823


## Tumor IoU

In [27]:
# Batch IoU over Tumor segmentation PNGs -> NIfTI per patient/slice
import os
import re
import numpy as np
from typing import Dict, Tuple, List

try:
    import nibabel as nib
except ImportError as e:
    raise ImportError("Missing dependency 'nibabel'. Install with: pip install nibabel") from e

try:
    from PIL import Image
except ImportError as e:
    raise ImportError("Missing dependency 'Pillow'. Install with: pip install pillow") from e

# Paths
root = os.getcwd()
seg_dir = os.path.join(root, "data", "Segmentation_Files")

# Try common tumor PNG folder names under LucaSegmentation
candidates = [
    os.path.join(root, "LucaSegmentation", "Tumor segmentation"),
    os.path.join(root, "LucaSegmentation", "Tumour segmentation"),
    os.path.join(root, "LucaSegmentation", "Tumor"),
    os.path.join(root, "LucaSegmentation", "Tumour"),
]

png_dir = next((p for p in candidates if os.path.isdir(p)), None)
print("Segmentation dir:", seg_dir)
print("Tumor PNG dir:", png_dir if png_dir else "<not found>")

if not os.path.isdir(seg_dir):
    raise FileNotFoundError(f"Segmentation directory not found: {seg_dir}")
if png_dir is None:
    print("No tumor PNG directory found in LucaSegmentation. Skipping tumor IoU batch.")
else:
    # Helpers
    fname_re = re.compile(r"tumou?r\s*seg\s*pat\s*(\d+)\s*slice\s*(\d+)\.png\Z", re.IGNORECASE)

    def parse_name(name: str) -> Tuple[int, int]:
        m = fname_re.search(name)
        if not m:
            raise ValueError(f"Filename not in expected format 'tumor seg pat <ID> slice <N>.png': {name}")
        return int(m.group(1)), int(m.group(2))

    # Binary IoU
    def compute_iou(a: np.ndarray, b: np.ndarray) -> float:
        inter = np.sum((a != 0) & (b != 0))
        union = np.sum((a != 0) | (b != 0))
        return float(inter) / float(union) if union > 0 else 0.0

    # Orientation variants to try
    transforms = {
        "identity": lambda x: x,
        "flipud": np.flipud,
        "fliplr": np.fliplr,
        "transpose": lambda x: x.T,
        "transpose_flipud": lambda x: np.flipud(x.T),
        "transpose_fliplr": lambda x: np.fliplr(x.T),
    }

    # Cache NIfTI volumes by patient id
    vol_cache: Dict[int, np.ndarray] = {}

    def load_volume_for_patient(pid: int) -> np.ndarray:
        if pid in vol_cache:
            return vol_cache[pid]
        nii_path = os.path.join(seg_dir, f"segmentation-{pid}.nii")
        if not os.path.exists(nii_path):
            raise FileNotFoundError(f"Missing NIfTI for patient {pid}: {nii_path}")
        img = nib.load(nii_path)
        img = nib.as_closest_canonical(img)
        data = img.get_fdata()
        vol = np.asarray(data, dtype=np.int16)
        if vol.ndim > 3:
            vol = np.squeeze(vol)
        if vol.ndim != 3:
            raise ValueError(f"Unexpected volume shape for patient {pid}: {vol.shape}")
        vol_cache[pid] = vol
        return vol

    # Discover PNGs
    png_files = [f for f in os.listdir(png_dir) if f.lower().endswith('.png')]
    png_files.sort()
    print(f"Found {len(png_files)} tumor PNG masks.")

    if not png_files:
        print("No PNG files found. Ensure the folder and naming are correct.")

    # Iterate and compute IoU per file, aggregate per patient
    per_file_results: List[Tuple[int, int, float, str]] = []  # (pid, slice, iou, orientation)
    per_patient_scores: Dict[int, List[float]] = {}

    for fname in png_files:
        try:
            pid, z = parse_name(fname)
        except ValueError as e:
            print("Skipping:", fname, "->", e)
            continue

        try:
            vol = load_volume_for_patient(pid)
        except Exception as e:
            print(f"Patient {pid}: failed to load volume -> {e}")
            continue

        if z < 0 or z >= vol.shape[2]:
            print(f"Patient {pid} slice {z}: out of bounds for Z={vol.shape[2]}")
            continue

        # Tumor GT is labels >= 2
        gt = (vol[:, :, z] >= 2).astype(np.uint8)

        # Load prediction and make binary
        png_path = os.path.join(png_dir, fname)
        with Image.open(png_path) as im:
            pred_arr = np.array(im.convert('L'), dtype=np.uint8)
        pred = (pred_arr >= 128).astype(np.uint8)

        # Resize prediction if needed
        if pred.shape != gt.shape:
            pred = (np.array(Image.fromarray(pred * 255).resize((gt.shape[1], gt.shape[0]), Image.NEAREST)) >= 128).astype(np.uint8)

        # Try orientations and pick best
        ious = {name: compute_iou(gt, fn(pred)) for name, fn in transforms.items()}
        best_name = max(ious, key=ious.get)
        best_iou = ious[best_name]

        per_file_results.append((pid, z, best_iou, best_name))
        per_patient_scores.setdefault(pid, []).append(best_iou)

    # Report
    print("\nTumor per-file results (first 20 shown):")
    for row in per_file_results[:20]:
        pid, z, iou, orient = row
        print(f"pat {pid:>3} slice {z:>4} -> IoU {iou:.4f} (orient={orient})")

    print("\nTumor per-patient summary:")
    for pid in sorted(per_patient_scores):
        vals = per_patient_scores[pid]
        mean_iou = float(np.mean(vals)) if vals else 0.0
        print(f"patient {pid:>3}: {len(vals)} slices, mean IoU {mean_iou:.4f}")


Segmentation dir: /Users/matteogiacomelli/Documents/Fisica_Magistrale/Image ProccesingOriginal/Project LiTS/data/Segmentation_Files
Tumor PNG dir: /Users/matteogiacomelli/Documents/Fisica_Magistrale/Image ProccesingOriginal/Project LiTS/LucaSegmentation/Tumor segmentation
Found 10 tumor PNG masks.

Tumor per-file results (first 20 shown):
pat   0 slice   62 -> IoU 0.0095 (orient=transpose_flipud)
pat  10 slice  380 -> IoU 0.4320 (orient=transpose_flipud)
pat  12 slice  400 -> IoU 0.0000 (orient=identity)
pat  16 slice  410 -> IoU 0.6406 (orient=transpose_flipud)
pat  18 slice  420 -> IoU 0.7242 (orient=transpose_flipud)
pat  24 slice  199 -> IoU 0.3968 (orient=transpose_flipud)
pat  27 slice  495 -> IoU 0.6328 (orient=transpose_flipud)
pat   4 slice  460 -> IoU 0.7176 (orient=transpose_flipud)
pat   6 slice  410 -> IoU 0.0000 (orient=identity)
pat   8 slice  476 -> IoU 0.0494 (orient=transpose_flipud)

Tumor per-patient summary:
patient   0: 1 slices, mean IoU 0.0095
patient   4: 1 sli

### Volume Overlapping Error Tumor

In [28]:
print("\nPer-patient tumor VOE summary:")
for pid in sorted(per_patient_scores):
    vals = per_patient_scores[pid]
    mean_iou = float(np.mean(vals)) if vals else 0.0
    print(f"patient {pid:>3}: {len(vals)} slices, mean VOE {1 - mean_iou:.4f}")


Per-patient tumor VOE summary:
patient   0: 1 slices, mean VOE 0.9905
patient   4: 1 slices, mean VOE 0.2824
patient   6: 1 slices, mean VOE 1.0000
patient   8: 1 slices, mean VOE 0.9506
patient  10: 1 slices, mean VOE 0.5680
patient  12: 1 slices, mean VOE 1.0000
patient  16: 1 slices, mean VOE 0.3594
patient  18: 1 slices, mean VOE 0.2758
patient  24: 1 slices, mean VOE 0.6032
patient  27: 1 slices, mean VOE 0.3672


# Liver Dice value

In [29]:
# Batch Dice over Liver segmentation PNGs -> NIfTI per patient/slice
import os
import re
import numpy as np
from typing import Dict, Tuple, List

try:
    import nibabel as nib
except ImportError as e:
    raise ImportError("Missing dependency 'nibabel'. Install with: pip install nibabel") from e

try:
    from PIL import Image
except ImportError as e:
    raise ImportError("Missing dependency 'Pillow'. Install with: pip install pillow") from e

# Paths
root = os.getcwd()
seg_dir = os.path.join(root, "data", "Segmentation_Files")
png_dir = os.path.join(root, "LucaSegmentation", "Liver segmentation")
print("Segmentation dir:", seg_dir)
print("PNG dir:", png_dir)

if not os.path.isdir(seg_dir):
    raise FileNotFoundError(f"Segmentation directory not found: {seg_dir}")
if not os.path.isdir(png_dir):
    raise FileNotFoundError(f"PNG directory not found: {png_dir}")

# Helpers
fname_re = re.compile(r"liver\s*seg\s*pat\s*(\d+)\s*slice\s*(\d+)\.png\Z", re.IGNORECASE)

def parse_name(name: str) -> Tuple[int, int]:
    m = fname_re.search(name)
    if not m:
        raise ValueError(f"Filename not in expected format 'liver seg pat <ID> slice <N>.png': {name}")
    return int(m.group(1)), int(m.group(2))

# Binary Dice

def compute_dice(a: np.ndarray, b: np.ndarray) -> float:
    inter = np.sum((a != 0) & (b != 0))
    a_sum = np.sum(a != 0)
    b_sum = np.sum(b != 0)
    denom = a_sum + b_sum
    return (2.0 * float(inter) / float(denom)) if denom > 0 else 0.0

# Orientation variants to try
transforms = {
    "identity": lambda x: x,
    "flipud": np.flipud,
    "fliplr": np.fliplr,
    "transpose": lambda x: x.T,
    "transpose_flipud": lambda x: np.flipud(x.T),
    "transpose_fliplr": lambda x: np.fliplr(x.T),
}

# Cache NIfTI volumes by patient id
vol_cache: Dict[int, np.ndarray] = {}

def load_volume_for_patient(pid: int) -> np.ndarray:
    if pid in vol_cache:
        return vol_cache[pid]
    nii_path = os.path.join(seg_dir, f"segmentation-{pid}.nii")
    if not os.path.exists(nii_path):
        raise FileNotFoundError(f"Missing NIfTI for patient {pid}: {nii_path}")
    img = nib.load(nii_path)
    img = nib.as_closest_canonical(img)
    data = img.get_fdata()
    vol = np.asarray(data, dtype=np.int16)
    if vol.ndim > 3:
        vol = np.squeeze(vol)
    if vol.ndim != 3:
        raise ValueError(f"Unexpected volume shape for patient {pid}: {vol.shape}")
    vol_cache[pid] = vol
    return vol

# Discover PNGs
png_files = [f for f in os.listdir(png_dir) if f.lower().endswith('.png')]
png_files.sort()
print(f"Found {len(png_files)} PNG masks.")

if not png_files:
    print("No PNG files found. Ensure the folder and naming are correct.")

# Iterate and compute Dice per file, aggregate per patient
per_file_results: List[Tuple[int, int, float, str]] = []  # (pid, slice, dice, orientation)
per_patient_scores: Dict[int, List[float]] = {}

for fname in png_files:
    try:
        pid, z = parse_name(fname)
    except ValueError as e:
        print("Skipping:", fname, "->", e)
        continue

    try:
        vol = load_volume_for_patient(pid)
    except Exception as e:
        print(f"Patient {pid}: failed to load volume -> {e}")
        continue

    if z < 0 or z >= vol.shape[2]:
        print(f"Patient {pid} slice {z}: out of bounds for Z={vol.shape[2]}")
        continue

    gt = (vol[:, :, z] >= 1).astype(np.uint8)

    # Load prediction and make binary
    png_path = os.path.join(png_dir, fname)
    with Image.open(png_path) as im:
        pred_arr = np.array(im.convert('L'), dtype=np.uint8)
    pred = (pred_arr >= 128).astype(np.uint8)

    # Resize prediction if needed
    if pred.shape != gt.shape:
        pred = (np.array(Image.fromarray(pred * 255).resize((gt.shape[1], gt.shape[0]), Image.NEAREST)) >= 128).astype(np.uint8)

    # Try orientations and pick best
    dices = {name: compute_dice(gt, fn(pred)) for name, fn in transforms.items()}
    best_name = max(dices, key=dices.get)
    best_dice = dices[best_name]

    per_file_results.append((pid, z, best_dice, best_name))
    per_patient_scores.setdefault(pid, []).append(best_dice)

# Report
print("\nPer-file Dice results (first 20 shown):")
for row in per_file_results[:20]:
    pid, z, dice, orient = row
    print(f"pat {pid:>3} slice {z:>4} -> Dice {dice:.4f} (orient={orient})")

print("\nPer-patient Dice summary:")
for pid in sorted(per_patient_scores):
    vals = per_patient_scores[pid]
    mean_dice = float(np.mean(vals)) if vals else 0.0
    print(f"patient {pid:>3}: {len(vals)} slices, mean Dice {mean_dice:.4f}")


Segmentation dir: /Users/matteogiacomelli/Documents/Fisica_Magistrale/Image ProccesingOriginal/Project LiTS/data/Segmentation_Files
PNG dir: /Users/matteogiacomelli/Documents/Fisica_Magistrale/Image ProccesingOriginal/Project LiTS/LucaSegmentation/Liver segmentation
Found 10 PNG masks.

Per-file Dice results (first 20 shown):
pat   0 slice   62 -> Dice 0.7770 (orient=transpose_flipud)
pat  10 slice  380 -> Dice 0.7321 (orient=transpose_flipud)
pat  12 slice  400 -> Dice 0.9321 (orient=transpose_flipud)
pat  16 slice  410 -> Dice 0.9113 (orient=transpose_flipud)
pat  18 slice  420 -> Dice 0.9405 (orient=transpose_flipud)
pat  24 slice  199 -> Dice 0.9131 (orient=transpose_flipud)
pat  27 slice  495 -> Dice 0.8997 (orient=transpose_flipud)
pat   4 slice  460 -> Dice 0.8968 (orient=transpose_flipud)
pat   6 slice  410 -> Dice 0.8148 (orient=transpose_flipud)
pat   8 slice  476 -> Dice 0.9487 (orient=transpose_flipud)

Per-patient Dice summary:
patient   0: 1 slices, mean Dice 0.7770
patie

## Tumor Dice
This section iterates over tumor segmentation PNGs in `LucaSegmentation/Tumor Segmentation`, extracts the patient ID and slice index from each filename (`tumor seg pat {ID} slice {N}.png` or `tumour seg ...`), loads the corresponding `segmentation-{ID}.nii`, and computes Dice for the specified slice. It tests common 2D orientation variants to account for PNG export rotations/flips, then reports per-file results and per-patient mean Dice.

In [30]:
# Batch Dice over Tumor segmentation PNGs -> NIfTI per patient/slice
import os
import re
import numpy as np
from typing import Dict, Tuple, List

try:
    import nibabel as nib
except ImportError as e:
    raise ImportError("Missing dependency 'nibabel'. Install with: pip install nibabel") from e

try:
    from PIL import Image
except ImportError as e:
    raise ImportError("Missing dependency 'Pillow'. Install with: pip install pillow") from e

# Paths
root = os.getcwd()
seg_dir = os.path.join(root, "data", "Segmentation_Files")
png_dir = os.path.join(root, "LucaSegmentation", "Tumor Segmentation")
print("Segmentation dir:", seg_dir)
print("Tumor PNG dir:", png_dir)

if not os.path.isdir(seg_dir):
    raise FileNotFoundError(f"Segmentation directory not found: {seg_dir}")
if not os.path.isdir(png_dir):
    raise FileNotFoundError(f"PNG directory not found: {png_dir}")

# Helpers
fname_re = re.compile(r"tumou?r\s*seg\s*pat\s*(\d+)\s*slice\s*(\d+)\.png\Z", re.IGNORECASE)

def parse_name(name: str) -> Tuple[int, int]:
    m = fname_re.search(name)
    if not m:
        raise ValueError(f"Filename not in expected format 'tumor seg pat <ID> slice <N>.png': {name}")
    return int(m.group(1)), int(m.group(2))

# Binary Dice

def compute_dice(a: np.ndarray, b: np.ndarray) -> float:
    inter = np.sum((a != 0) & (b != 0))
    a_sum = np.sum(a != 0)
    b_sum = np.sum(b != 0)
    denom = a_sum + b_sum
    return (2.0 * float(inter) / float(denom)) if denom > 0 else 0.0

# Orientation variants to try
transforms = {
    "identity": lambda x: x,
    "flipud": np.flipud,
    "fliplr": np.fliplr,
    "transpose": lambda x: x.T,
    "transpose_flipud": lambda x: np.flipud(x.T),
    "transpose_fliplr": lambda x: np.fliplr(x.T),
}

# Cache NIfTI volumes by patient id
vol_cache: Dict[int, np.ndarray] = {}

def load_volume_for_patient(pid: int) -> np.ndarray:
    if pid in vol_cache:
        return vol_cache[pid]
    nii_path = os.path.join(seg_dir, f"segmentation-{pid}.nii")
    if not os.path.exists(nii_path):
        raise FileNotFoundError(f"Missing NIfTI for patient {pid}: {nii_path}")
    img = nib.load(nii_path)
    img = nib.as_closest_canonical(img)
    data = img.get_fdata()
    vol = np.asarray(data, dtype=np.int16)
    if vol.ndim > 3:
        vol = np.squeeze(vol)
    if vol.ndim != 3:
        raise ValueError(f"Unexpected volume shape for patient {pid}: {vol.shape}")
    vol_cache[pid] = vol
    return vol

# Discover PNGs
png_files = [f for f in os.listdir(png_dir) if f.lower().endswith('.png')]
png_files.sort()
print(f"Found {len(png_files)} tumor PNG masks.")

if not png_files:
    print("No PNG files found. Ensure the folder and naming are correct.")

# Iterate and compute Dice per file, aggregate per patient
per_file_results: List[Tuple[int, int, float, str]] = []  # (pid, slice, dice, orientation)
per_patient_scores: Dict[int, List[float]] = {}

for fname in png_files:
    try:
        pid, z = parse_name(fname)
    except ValueError as e:
        print("Skipping:", fname, "->", e)
        continue

    try:
        vol = load_volume_for_patient(pid)
    except Exception as e:
        print(f"Patient {pid}: failed to load volume -> {e}")
        continue

    if z < 0 or z >= vol.shape[2]:
        print(f"Patient {pid} slice {z}: out of bounds for Z={vol.shape[2]}")
        continue

    # Tumor GT is labels >= 2
    gt = (vol[:, :, z] >= 2).astype(np.uint8)

    # Load prediction and make binary
    png_path = os.path.join(png_dir, fname)
    with Image.open(png_path) as im:
        pred_arr = np.array(im.convert('L'), dtype=np.uint8)
    pred = (pred_arr >= 128).astype(np.uint8)

    # Resize prediction if needed
    if pred.shape != gt.shape:
        pred = (np.array(Image.fromarray(pred * 255).resize((gt.shape[1], gt.shape[0]), Image.NEAREST)) >= 128).astype(np.uint8)

    # Try orientations and pick best
    dices = {name: compute_dice(gt, fn(pred)) for name, fn in transforms.items()}
    best_name = max(dices, key=dices.get)
    best_dice = dices[best_name]

    per_file_results.append((pid, z, best_dice, best_name))
    per_patient_scores.setdefault(pid, []).append(best_dice)

# Report
print("\nTumor per-file Dice results (first 20 shown):")
for row in per_file_results[:20]:
    pid, z, dice, orient = row
    print(f"pat {pid:>3} slice {z:>4} -> Dice {dice:.4f} (orient={orient})")

print("\nTumor per-patient Dice summary:")
for pid in sorted(per_patient_scores):
    vals = per_patient_scores[pid]
    mean_dice = float(np.mean(vals)) if vals else 0.0
    print(f"patient {pid:>3}: {len(vals)} slices, mean Dice {mean_dice:.4f}")


Segmentation dir: /Users/matteogiacomelli/Documents/Fisica_Magistrale/Image ProccesingOriginal/Project LiTS/data/Segmentation_Files
Tumor PNG dir: /Users/matteogiacomelli/Documents/Fisica_Magistrale/Image ProccesingOriginal/Project LiTS/LucaSegmentation/Tumor Segmentation
Found 10 tumor PNG masks.



Tumor per-file Dice results (first 20 shown):
pat   0 slice   62 -> Dice 0.0188 (orient=transpose_flipud)
pat  10 slice  380 -> Dice 0.6033 (orient=transpose_flipud)
pat  12 slice  400 -> Dice 0.0000 (orient=identity)
pat  16 slice  410 -> Dice 0.7809 (orient=transpose_flipud)
pat  18 slice  420 -> Dice 0.8401 (orient=transpose_flipud)
pat  24 slice  199 -> Dice 0.5681 (orient=transpose_flipud)
pat  27 slice  495 -> Dice 0.7751 (orient=transpose_flipud)
pat   4 slice  460 -> Dice 0.8356 (orient=transpose_flipud)
pat   6 slice  410 -> Dice 0.0000 (orient=identity)
pat   8 slice  476 -> Dice 0.0941 (orient=transpose_flipud)

Tumor per-patient Dice summary:
patient   0: 1 slices, mean Dice 0.0188
patient   4: 1 slices, mean Dice 0.8356
patient   6: 1 slices, mean Dice 0.0000
patient   8: 1 slices, mean Dice 0.0941
patient  10: 1 slices, mean Dice 0.6033
patient  12: 1 slices, mean Dice 0.0000
patient  16: 1 slices, mean Dice 0.7809
patient  18: 1 slices, mean Dice 0.8401
patient  24: 1 s

# Relative Volume Difference

## Tumor RVD
Compute RVD for tumor segmentation PNGs in `LucaSegmentation/Tumor Segmentation` against `segmentation-{ID}.nii` tumors (labels >= 2). For each slice, try common 2D orientation variants and pick the orientation that minimizes |RVD|. Report per-file RVD and per-patient mean RVD.

In [31]:
# Batch RVD over Tumor segmentation PNGs -> NIfTI per patient/slice
import os
import re
import numpy as np
from typing import Dict, Tuple, List

try:
    import nibabel as nib
except ImportError as e:
    raise ImportError("Missing dependency 'nibabel'. Install with: pip install nibabel") from e

try:
    from PIL import Image
except ImportError as e:
    raise ImportError("Missing dependency 'Pillow'. Install with: pip install pillow") from e

# Paths
root = os.getcwd()
seg_dir = os.path.join(root, "data", "Segmentation_Files")
png_dir = os.path.join(root, "LucaSegmentation", "Tumor Segmentation")
print("Segmentation dir:", seg_dir)
print("Tumor PNG dir:", png_dir)

if not os.path.isdir(seg_dir):
    raise FileNotFoundError(f"Segmentation directory not found: {seg_dir}")
if not os.path.isdir(png_dir):
    raise FileNotFoundError(f"PNG directory not found: {png_dir}")

# Helpers
fname_re = re.compile(r"tumou?r\s*seg\s*pat\s*(\d+)\s*slice\s*(\d+)\.png\Z", re.IGNORECASE)

def parse_name(name: str) -> Tuple[int, int]:
    m = fname_re.search(name)
    if not m:
        raise ValueError(f"Filename not in expected format 'tumor seg pat <ID> slice <N>.png': {name}")
    return int(m.group(1)), int(m.group(2))

# RVD

def compute_rvd(a: np.ndarray, b: np.ndarray) -> float:
    a_sum = np.sum(a != 0)
    b_sum = np.sum(b != 0)
    return (float(a_sum) - float(b_sum)) / float(b_sum) if b_sum > 0 else 0.0

# Orientation variants to try
transforms = {
    "identity": lambda x: x,
    "flipud": np.flipud,
    "fliplr": np.fliplr,
    "transpose": lambda x: x.T,
    "transpose_flipud": lambda x: np.flipud(x.T),
    "transpose_fliplr": lambda x: np.fliplr(x.T),
}

# Cache NIfTI volumes by patient id
vol_cache: Dict[int, np.ndarray] = {}

def load_volume_for_patient(pid: int) -> np.ndarray:
    if pid in vol_cache:
        return vol_cache[pid]
    nii_path = os.path.join(seg_dir, f"segmentation-{pid}.nii")
    if not os.path.exists(nii_path):
        raise FileNotFoundError(f"Missing NIfTI for patient {pid}: {nii_path}")
    img = nib.load(nii_path)
    img = nib.as_closest_canonical(img)
    data = img.get_fdata()
    vol = np.asarray(data, dtype=np.int16)
    if vol.ndim > 3:
        vol = np.squeeze(vol)
    if vol.ndim != 3:
        raise ValueError(f"Unexpected volume shape for patient {pid}: {vol.shape}")
    vol_cache[pid] = vol
    return vol

# Discover PNGs
png_files = [f for f in os.listdir(png_dir) if f.lower().endswith('.png')]
png_files.sort()
print(f"Found {len(png_files)} tumor PNG masks.")

if not png_files:
    print("No PNG files found. Ensure the folder and naming are correct.")

# Iterate and compute RVD per file, aggregate per patient
per_file_results: List[Tuple[int, int, float, str]] = []  # (pid, slice, rvd, orientation)
per_patient_scores: Dict[int, List[float]] = {}

for fname in png_files:
    try:
        pid, z = parse_name(fname)
    except ValueError as e:
        print("Skipping:", fname, "->", e)
        continue

    try:
        vol = load_volume_for_patient(pid)
    except Exception as e:
        print(f"Patient {pid}: failed to load volume -> {e}")
        continue

    if z < 0 or z >= vol.shape[2]:
        print(f"Patient {pid} slice {z}: out of bounds for Z={vol.shape[2]}")
        continue

    # Tumor GT is labels >= 2
    gt = (vol[:, :, z] >= 2).astype(np.uint8)

    # Load prediction and make binary
    png_path = os.path.join(png_dir, fname)
    with Image.open(png_path) as im:
        pred_arr = np.array(im.convert('L'), dtype=np.uint8)
    pred = (pred_arr >= 128).astype(np.uint8)

    # Resize prediction if needed
    if pred.shape != gt.shape:
        pred = (np.array(Image.fromarray(pred * 255).resize((gt.shape[1], gt.shape[0]), Image.NEAREST)) >= 128).astype(np.uint8)

    # Try orientations and pick the one minimizing |RVD|
    rvd_scores = {name: compute_rvd(gt, fn(pred)) for name, fn in transforms.items()}
    best_name = min(rvd_scores, key=lambda k: abs(rvd_scores[k]))
    best_rvd = rvd_scores[best_name]

    per_file_results.append((pid, z, best_rvd, best_name))
    per_patient_scores.setdefault(pid, []).append(best_rvd)

# Report
print("\nTumor per-file RVD results (first 20 shown):")
for row in per_file_results[:20]:
    pid, z, rvd, orient = row
    print(f"pat {pid:>3} slice {z:>4} -> RVD {rvd:+.4f} (orient={orient})")

print("\nTumor per-patient RVD summary:")
for pid in sorted(per_patient_scores):
    vals = per_patient_scores[pid]
    mean_rvd = float(np.mean(vals)) if vals else 0.0
    print(f"patient {pid:>3}: {len(vals)} slices, mean RVD {mean_rvd:+.4f}")


Segmentation dir: /Users/matteogiacomelli/Documents/Fisica_Magistrale/Image ProccesingOriginal/Project LiTS/data/Segmentation_Files
Tumor PNG dir: /Users/matteogiacomelli/Documents/Fisica_Magistrale/Image ProccesingOriginal/Project LiTS/LucaSegmentation/Tumor Segmentation
Found 10 tumor PNG masks.



Tumor per-file RVD results (first 20 shown):
pat   0 slice   62 -> RVD -0.9905 (orient=identity)
pat  10 slice  380 -> RVD -0.1417 (orient=identity)
pat  12 slice  400 -> RVD +0.0000 (orient=identity)
pat  16 slice  410 -> RVD +0.3425 (orient=identity)
pat  18 slice  420 -> RVD +0.0000 (orient=identity)
pat  24 slice  199 -> RVD -0.1896 (orient=identity)
pat  27 slice  495 -> RVD +0.2669 (orient=identity)
pat   4 slice  460 -> RVD +0.3883 (orient=identity)
pat   6 slice  410 -> RVD +0.0000 (orient=identity)
pat   8 slice  476 -> RVD -0.9373 (orient=identity)

Tumor per-patient RVD summary:
patient   0: 1 slices, mean RVD -0.9905
patient   4: 1 slices, mean RVD +0.3883
patient   6: 1 slices, mean RVD +0.0000
patient   8: 1 slices, mean RVD -0.9373
patient  10: 1 slices, mean RVD -0.1417
patient  12: 1 slices, mean RVD +0.0000
patient  16: 1 slices, mean RVD +0.3425
patient  18: 1 slices, mean RVD +0.0000
patient  24: 1 slices, mean RVD -0.1896
patient  27: 1 slices, mean RVD +0.2669


## Liver RVD
Compute RVD for liver segmentation PNGs in `LucaSegmentation/Liver segmentation` against `segmentation-{ID}.nii` liver (labels >= 1). For each slice, try common 2D orientation variants and pick the orientation that minimizes |RVD|. Report per-file RVD and per-patient mean RVD.

In [32]:
# Batch RVD over Liver segmentation PNGs -> NIfTI per patient/slice
import os
import re
import numpy as np
from typing import Dict, Tuple, List

try:
    import nibabel as nib
except ImportError as e:
    raise ImportError("Missing dependency 'nibabel'. Install with: pip install nibabel") from e

try:
    from PIL import Image
except ImportError as e:
    raise ImportError("Missing dependency 'Pillow'. Install with: pip install pillow") from e

# Paths
root = os.getcwd()
seg_dir = os.path.join(root, "data", "Segmentation_Files")
png_dir = os.path.join(root, "LucaSegmentation", "Liver segmentation")
print("Segmentation dir:", seg_dir)
print("Liver PNG dir:", png_dir)

if not os.path.isdir(seg_dir):
    raise FileNotFoundError(f"Segmentation directory not found: {seg_dir}")
if not os.path.isdir(png_dir):
    raise FileNotFoundError(f"PNG directory not found: {png_dir}")

# Helpers
fname_re = re.compile(r"liver\s*seg\s*pat\s*(\d+)\s*slice\s*(\d+)\.png\Z", re.IGNORECASE)

def parse_name(name: str) -> Tuple[int, int]:
    m = fname_re.search(name)
    if not m:
        raise ValueError(f"Filename not in expected format 'liver seg pat <ID> slice <N>.png': {name}")
    return int(m.group(1)), int(m.group(2))

# RVD

def compute_rvd(a: np.ndarray, b: np.ndarray) -> float:
    a_sum = np.sum(a != 0)
    b_sum = np.sum(b != 0)
    return (float(a_sum) - float(b_sum)) / float(b_sum) if b_sum > 0 else 0.0

# Orientation variants to try
transforms = {
    "identity": lambda x: x,
    "flipud": np.flipud,
    "fliplr": np.fliplr,
    "transpose": lambda x: x.T,
    "transpose_flipud": lambda x: np.flipud(x.T),
    "transpose_fliplr": lambda x: np.fliplr(x.T),
}

# Cache NIfTI volumes by patient id
vol_cache: Dict[int, np.ndarray] = {}

def load_volume_for_patient(pid: int) -> np.ndarray:
    if pid in vol_cache:
        return vol_cache[pid]
    nii_path = os.path.join(seg_dir, f"segmentation-{pid}.nii")
    if not os.path.exists(nii_path):
        raise FileNotFoundError(f"Missing NIfTI for patient {pid}: {nii_path}")
    img = nib.load(nii_path)
    img = nib.as_closest_canonical(img)
    data = img.get_fdata()
    vol = np.asarray(data, dtype=np.int16)
    if vol.ndim > 3:
        vol = np.squeeze(vol)
    if vol.ndim != 3:
        raise ValueError(f"Unexpected volume shape for patient {pid}: {vol.shape}")
    vol_cache[pid] = vol
    return vol

# Discover PNGs
png_files = [f for f in os.listdir(png_dir) if f.lower().endswith('.png')]
png_files.sort()
print(f"Found {len(png_files)} liver PNG masks.")

if not png_files:
    print("No PNG files found. Ensure the folder and naming are correct.")

# Iterate and compute RVD per file, aggregate per patient
per_file_results: List[Tuple[int, int, float, str]] = []  # (pid, slice, rvd, orientation)
per_patient_scores: Dict[int, List[float]] = {}

for fname in png_files:
    try:
        pid, z = parse_name(fname)
    except ValueError as e:
        print("Skipping:", fname, "->", e)
        continue

    try:
        vol = load_volume_for_patient(pid)
    except Exception as e:
        print(f"Patient {pid}: failed to load volume -> {e}")
        continue

    if z < 0 or z >= vol.shape[2]:
        print(f"Patient {pid} slice {z}: out of bounds for Z={vol.shape[2]}")
        continue

    # Liver GT is labels >= 1
    gt = (vol[:, :, z] >= 1).astype(np.uint8)

    # Load prediction and make binary
    png_path = os.path.join(png_dir, fname)
    with Image.open(png_path) as im:
        pred_arr = np.array(im.convert('L'), dtype=np.uint8)
    pred = (pred_arr >= 128).astype(np.uint8)

    # Resize prediction if needed
    if pred.shape != gt.shape:
        pred = (np.array(Image.fromarray(pred * 255).resize((gt.shape[1], gt.shape[0]), Image.NEAREST)) >= 128).astype(np.uint8)

    # Try orientations and pick the one minimizing |RVD|
    rvd_scores = {name: compute_rvd(gt, fn(pred)) for name, fn in transforms.items()}
    best_name = min(rvd_scores, key=lambda k: abs(rvd_scores[k]))
    best_rvd = rvd_scores[best_name]

    per_file_results.append((pid, z, best_rvd, best_name))
    per_patient_scores.setdefault(pid, []).append(best_rvd)

# Report
print("\nLiver per-file RVD results (first 20 shown):")
for row in per_file_results[:20]:
    pid, z, rvd, orient = row
    print(f"pat {pid:>3} slice {z:>4} -> RVD {rvd:+.4f} (orient={orient})")

print("\nLiver per-patient RVD summary:")
for pid in sorted(per_patient_scores):
    vals = per_patient_scores[pid]
    mean_rvd = float(np.mean(vals)) if vals else 0.0
    print(f"patient {pid:>3}: {len(vals)} slices, mean RVD {mean_rvd:+.4f}")


Segmentation dir: /Users/matteogiacomelli/Documents/Fisica_Magistrale/Image ProccesingOriginal/Project LiTS/data/Segmentation_Files
Liver PNG dir: /Users/matteogiacomelli/Documents/Fisica_Magistrale/Image ProccesingOriginal/Project LiTS/LucaSegmentation/Liver segmentation
Found 10 liver PNG masks.

Liver per-file RVD results (first 20 shown):
pat   0 slice   62 -> RVD +0.2461 (orient=identity)
pat  10 slice  380 -> RVD -0.3947 (orient=identity)
pat  12 slice  400 -> RVD -0.0140 (orient=identity)
pat  16 slice  410 -> RVD -0.0367 (orient=identity)
pat  18 slice  420 -> RVD +0.1084 (orient=identity)
pat  24 slice  199 -> RVD +0.1774 (orient=identity)
pat  27 slice  495 -> RVD -0.1465 (orient=identity)
pat   4 slice  460 -> RVD -0.0307 (orient=identity)
pat   6 slice  410 -> RVD -0.2844 (orient=identity)
pat   8 slice  476 -> RVD -0.0213 (orient=identity)

Liver per-patient RVD summary:
patient   0: 1 slices, mean RVD +0.2461
patient   4: 1 slices, mean RVD -0.0307
patient   6: 1 slices, 